# Определение эмоционального окраса сообщений в чате FINOPOLIS

__Задача__— создать прототип сервиса, который с помощью машинного обучения сможет определять эмоциональный окрас входящих сообщений и классифицировать их по категориям.

Протестировать его работоспособность на открытых данных, используя датасет с отзывами о фильмах, Large Movie Dataset или Sentiment140 от Стэндфордского университета, а также на реальных кейсах, собранных Почта Банком.

## 2. Исследование моделей

На предыдущем шаге была произведена обработка и очистка датасета __Sentiment140__, теперь на данных можно обучать модели.

Перед нами стоит задача классификации текста на две группы в зависимости от эмоциональной окраски: 0 - negative, 1 - positive. 

Для классификации могут использоваться следующие модели:
* Линейные классификаторы
* Решающие деревья 
* Байесовские классификаторы 
*  Нейронные сети 


Для кодирования слов в тексте использовалась следующая последовательность методов: CountVectorizer()+ TfidfTransformer()

In [102]:
# импорт библиотек
import numpy as np
import pandas as pd
import json
import re

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


from sklearn.utils import shuffle
from sklearn.base import TransformerMixin
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from random import choice

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### 2.1 Загрузка данных

Загрузим предобработанные на предыдущем шаге данные

In [ ]:
train = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')

In [ ]:
# обучающий датасет
train.head() 

,label,text
0,0,awww that s a bummer you shoulda got david car...
1,0,is upset that he can t update his facebook by ...
2,0,i dived many times for the ball managed to sav...
3,0,my whole body feels itchy and like its on fire
4,0,no it s not behaving at all i m mad why am i h...


In [ ]:
# тестовая выборка
test.head()

,label,text
0,1,i loooooooovvvvvveee my kindle not that the dx...
1,1,reading my kindle love it lee childs is good read
2,1,ok first assesment of the kindle it fucking rocks
3,1,you ll love your kindle i ve had mine for a fe...
4,1,fair enough but i have the kindle and i think ...


In [ ]:
X_train = train['text'].values
y_train = train['label'].values
X_test = test['text'].values
y_test = test['label'].values

### 2.2 Обучение моделей

In [ ]:
# вспомогательная функция, для упроцения пайплайна обучения
def make_pipeline(vectorizer, transformer, classifier):
    return Pipeline([
            ('vectorizer', vectorizer),
            ('transformer', transformer),
            ('classifier', classifier)])

### 2.2.1 Линейные классификаторы

In [ ]:
%%time
for name, clf in {'LogisticRegression': LogisticRegression, 'LinearSVC': LinearSVC, 
               'SGDClassifier': SGDClassifier, 'RidgeClassifier': RidgeClassifier}.items():
    score = cross_val_score(make_pipeline(CountVectorizer(), TfidfTransformer(), clf(random_state=777)), X_train, y_train, cv=5).mean()
    print(f"{name} - {score}")

LogisticRegression - 0.7934825
LinearSVC - 0.785765625
SGDClassifier - 0.77611375
RidgeClassifier - 0.7851600000000001
CPU times: user 17min 39s, sys: 4min 22s, total: 22min 2s
Wall time: 17min 14s


Таким образом, на кросс-валидации лучшие результаты показали линейные модели LogisticRegression и LinearSVC, попробуем подобрать параметры для данных моделей.

#### Настройка параметров для линейных моделей


In [ ]:
stop_words =  stopwords.words('english')

In [ ]:
params_grid_vectorizer = {
    'vectorizer__max_df' : [0.85, 0.9, 1.0],
    'vectorizer__min_df' : [1, 10, 20], 
    'vectorizer__ngram_range' : [(1, 1), (1, 2), (1, 3), (1, 5)],
    'vectorizer__stop_words' : [stop_words, None, 'english']
}

In [ ]:
params_grid_lr = {
    'classifier__C': [0.15, 0.3, 0.5, 0.8],
    'classifier__max_iter': [100, 300, 500],
    'classifier__solver': ['lbfgs', 'liblinear', 'sag']
}
params_grid_lsvc = {
    'classifier__loss': ['hinge', 'squared_hinge'], 
    'classifier__max_iter': [100, 300, 500],
    'classifier__tol': [1e-5, 1e-4, 1e-3],
    'classifier__C': [0.15, 0.3, 0.5, 0.8]
}

In [ ]:
def make_estimator(classifier, params_grid, scorer, data, labels):
    pipeline = make_pipeline(CountVectorizer(), TfidfTransformer(), classifier)
    grid_cv = RandomizedSearchCV(pipeline, params_grid, scoring=scorer, cv=5, random_state=777, n_iter=100)
    grid_cv.fit(data, labels)
    return grid_cv

In [ ]:
grid_search_lr = make_estimator(LogisticRegression(random_state=777), 
                                {**params_grid_vectorizer, **params_grid_lr}, 'accuracy', X_train, y_train)
print("LogisticRegression:")
print(f"Лучшие параметры - {grid_search_lr.best_params_}")

LogisticRegression:
Лучшие параметры - {'vectorizer__stop_words': None, 'vectorizer__ngram_range': (1, 1), 'vectorizer__min_df': 1, 'vectorizer__max_df': 1.0, 'classifier__solver': 'lbfgs', 'classifier__max_iter': 300, 'classifier__C': 0.5}


In [ ]:
grid_search_lsvc = make_estimator(LinearSVC(random_state=777), 
                                  {**params_grid_vectorizer, **params_grid_lsvc}, 'accuracy', X_train, y_train)
print("LinearSVC:")
print(f"Параметры - {grid_search_lsvc.best_params_}")

LinearSVC:
Параметры - {'vectorizer__stop_words': None, 'vectorizer__ngram_range': (1, 2), 'vectorizer__min_df': 1, 'vectorizer__max_df': 1.0, 'classifier__tol': 1e-05, 'classifier__max_iter': 500, 'classifier__loss': 'hinge', 'classifier__C': 0.8}


In [ ]:
results_linear = {'LogisticRegression': (grid_search_lr.best_score_, grid_search_lr.best_params_),
                  'LinearSVC': (grid_search_lsvc.best_score_, grid_search_lsvc.best_params_),
                  }
print(f"Лучшая линейная модель - {max(results_linear, key=results_linear.get)}")
print(f"Параметры - {max(results_linear.values())[1]}")

Лучшая линейная модель - LogisticRegression
Параметры - {'vectorizer__stop_words': None, 'vectorizer__ngram_range': (1, 1), 'vectorizer__min_df': 1, 'vectorizer__max_df': 1.0, 'classifier__solver': 'lbfgs', 'classifier__max_iter': 300, 'classifier__C': 0.5}


#### Качество лучшей линейной модели с подбором параметров

In [ ]:
best_lin_model = Pipeline([
                           ('vectorizer', CountVectorizer(min_df=1, ngram_range=(1, 1), max_df=1.0, stop_words=None)),
                           ('transformer', TfidfTransformer()),
                           ('classifier', LogisticRegression(max_iter=300, solver='lbfgs', C=0.5,  random_state=777))
                           ])

In [ ]:
best_lin_model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabula...
                ('transformer',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 LogisticRegression(C=0.5, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_s

In [ ]:
best_lin_model.score(X_train, y_train)

0.809895625

In [ ]:
best_lin_model.score(X_test, y_test)

0.8189415041782729



---



### 2.2.2 Байесовские классификаторы

In [ ]:
%%time
for name, clf in {'MultinomialNB': MultinomialNB, 'BernoulliNB': BernoulliNB}.items():
    score = cross_val_score(make_pipeline(CountVectorizer(), TfidfTransformer(), clf()), X_train, y_train, cv=5).mean()
    print(f"{name} - {score}")

MultinomialNB - 0.7734399999999999
BernoulliNB - 0.780085625
CPU times: user 4min 15s, sys: 1.98 s, total: 4min 17s
Wall time: 4min 20s


#### Настройка параметров

In [ ]:
params_grid_mnb = {
    'classifier__alpha': np.logspace(0, 5, 10), 
    'classifier__fit_prior': [True, False]
}
params_grid_bnb = {
    'classifier__alpha': np.logspace(0, 5, 10),
    'classifier__fit_prior': [True, False]
}

In [ ]:
grid_search_bnb = make_estimator(BernoulliNB(), 
                                {**params_grid_vectorizer, **params_grid_bnb}, 'accuracy', X_train, y_train)
print("BernoulliNB:")
print(f"Параметры - {grid_search_bnb.best_params_}")

BernoulliNB:
Параметры - {'vectorizer__stop_words': ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 

In [ ]:
results_bayes = {'BernoulliNB': (grid_search_bnb.best_score_, grid_search_bnb.best_params_)}
print(f"Лучшая байесовская модель - {max(results_bayes, key=results_bayes.get)}")
print(f"Параметры - {max(results_bayes.values())[1]}")

Лучшая байесовская модель - BernoulliNB
Параметры - {'vectorizer__stop_words': ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'oth

#### Качество лучшей линейной модели с подбором параметров

In [ ]:
best_bayes_model = Pipeline([
                           ('vectorizer', CountVectorizer(min_df=1, ngram_range=(1, 2), max_df=1.0, stop_words=stop_words)),
                           ('transformer', TfidfTransformer()),
                           ('classifier', BernoulliNB(fit_prior=False, alpha=12.9))
                           ])

In [ ]:
best_bayes_model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "yo...
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...],
                                 strip_accents=None,
             

In [ ]:
best_bayes_model.score(X_train, y_train)

0.815718125

In [ ]:
best_bayes_model.score(X_test, y_test)

0.8495821727019499



---




### 2.2.3. Решающие деревья

In [ ]:
%%time
for name, clf in {'DecisionTreeClassifier': DecisionTreeClassifier, 'RandomForestClassifier': RandomForestClassifier, 
               'BaggingClassifier': BaggingClassifier, 'GradientBoostingClassifier': GradientBoostingClassifier}.items():
    score = cross_val_score(make_pipeline(CountVectorizer(), TfidfTransformer(), clf(random_state=777)), X_train, y_train, cv=5).mean()
    print(f"{name} - {score}")

DecisionTreeClassifier - 0.6704666666666667
RandomForestClassifier - 0.7521
BaggingClassifier - 0.7079333333333333
GradientBoostingClassifier - 0.6963666666666667
CPU times: user 11min, sys: 790 ms, total: 11min
Wall time: 10min 57s


In [ ]:
rfc = Pipeline([
                ('vectorizer', CountVectorizer(min_df=1, ngram_range=(1, 2), max_df=1.0, stop_words=stop_words)),
                ('transformer', TfidfTransformer()),
                ('classifier', RandomForestClassifier())
                ])

In [ ]:
rfc.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "yo...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
 

In [ ]:
rfc.score(X_test, y_test)

0.7103064066852368

Модели на основе решающих деревьев показывают результаты на валидации хуже, чем линейные модели, более того, данный тип моделей в принципе хуже справляется с классификацией текста из-за особенностей построения деревьев, поэтому производить подбор параметров для данных моделей не будем.


## 2.3 Улучшение модели

На этапе обучения и тестирования лучше всего показала себя __модель логистической регрессии__:
*   accuracy train: 0.8098
*   accuracy train: 0.8189



__Однако, необходимо протестировать модель на кейсах, предложенных Почтабанком__.

В текущем виде модель обучена на английском корпусе слов, что не позволит корректно отработать на кейсах Почтабанка, поэтому дообучим модель дополнительно на русском корпусе слов. Метки классов: negative, positive, neutral

Более того, в поставленной задаче нам важно, чтобы текст классифицировался не только на два класса: __положительный и негативный__, так как зачастую сообщения в чате могут носить безэмоциональный характер. Поэтому для обучения модели на русском корпусе слов введем ещё один класс: __нейтральный__.

Для обучения возмем датасет, состоящий из сводок новостных сообщений на русском языке.

In [70]:
with open('train.json') as f:
    raw_train = json.load(f)

In [71]:
data_train = pd.json_normalize(raw_train).drop(columns=['id'])
data_train.columns = ['text', 'label']
data_train.head()

,text,label
0,Досудебное расследование по факту покупки ЕНПФ...,negative
1,Медики рассказали о состоянии пострадавшего му...,negative
2,"Прошел почти год, как железнодорожным оператор...",negative
3,По итогам 12 месяцев 2016 года на территории р...,negative
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,negative


In [73]:
train_x = data_train['text'].values
train_y = data_train['label'].values

In [74]:
params = {'stop_words':stopwords.words(['russian', 'english']), 'ngram_range': (1, 3), 'min_df': 3 }
tfidf  = TfidfVectorizer(**params)

In [75]:
tfidf.fit(data_train['text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=3, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с',
                            'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                            'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
                            'по', 'только', 'ее', 'мне', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [111]:
m_params = {'solver':'lbfgs', 'multi_class': 'multinomial'}
model = LogisticRegression(**m_params)

In [113]:
model.fit(tfidf.transform(train_x), train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [114]:
model.score(tfidf.transform(train_x), train_y)

0.9296560272699101

## Протестируем обученную модель на кейсах Почтабанка

In [115]:
test_pochtabank = pd.read_excel('test_pochtabank.xlsx')
test_pochtabank.head()

,label,text
0,0,Как только я заинтересовался распечаткой движе...
1,0,Добрый день. Почему у меня просроченная задолж...
2,1,Всё о заявке на кредит. Как оставить заявку? П...
3,0,"Здравствуйте, я произвела частичное досрочное ..."
4,1,Как удалить старую карту. Оценка консультации ...


In [116]:
test_pochtabank['label'] = test_pochtabank['label'].replace({0:'negative', 1:'neutral', 2:'positive'})

In [117]:
x_test = test_pochtabank['text'].values
y_test = test_pochtabank['label'].values

In [118]:
pred = model.predict(tfidf.transform(x_test))

In [119]:
accuracy_score(y_test, pred)

0.8571428571428571

In [122]:
test_pochtabank['prediction'] = pred

In [124]:
test_pochtabank = test_pochtabank[['text', 'label', 'prediction']]
test_pochtabank.head()

,text,label,prediction
0,Как только я заинтересовался распечаткой движе...,negative,negative
1,Добрый день. Почему у меня просроченная задолж...,negative,negative
2,Всё о заявке на кредит. Как оставить заявку? П...,neutral,neutral
3,"Здравствуйте, я произвела частичное досрочное ...",negative,neutral
4,Как удалить старую карту. Оценка консультации ...,neutral,neutral


Посмотрим, на каком кейсе была допущена ошибка:

In [135]:
test_pochtabank.loc[test_pochtabank['label']!= test_pochtabank['prediction']]['text'].values

array(['Здравствуйте, я произвела частичное досрочное погашение кредита, но у меня уменьшился срок кредита, а не перерасчёт процентов, как сделать чтобы был именно платёж уменьшался? Нужен специалист. Ясно, увы вы опять меня разочаровали, это последнее наше сотрудничество Оценка консультации - 1 из 5'],
      dtype=object)

При необходимости можно оценивать вероятности принадлежности тому или иному классу:

In [131]:
model.predict_proba(tfidf.transform(x_test))

array([[0.45460102, 0.34788451, 0.19751448],
       [0.48135622, 0.32413306, 0.19451072],
       [0.34088063, 0.37083391, 0.28828546],
       [0.22218874, 0.48674657, 0.29106469],
       [0.23040849, 0.41137912, 0.3582124 ],
       [0.1940125 , 0.36085805, 0.44512945],
       [0.2022444 , 0.49411714, 0.30363846]])

In [125]:
test_pochtabank.to_excel('prediction_pochtabank.xlsx')

Таким образом, на кейсах Почтабанка модель показывает достойные результаты и ошибается лишь в одном случае, классифицируя неявное негативное обращение к нейтральному.